# Задание 6

### 6. Случайная величина имеет распределение Парето.

\begin{equation*} 
p(x) =
\begin{cases} 
\frac{\theta - 1}{x^{\theta}}, & \text{если } x \geq 1 \\
0, & \text{если } x < 1
\end{cases}
,\theta > 1.
\end{equation*}


In [1]:
import numpy as np
import random as rd

#### d)  Сгенеруйте выборку объёму n = 100 для некоторого значения парамeтра $\theta$. Вычислите указанные выше доверительные интегралы для доверительной веротяности 0.95.

In [2]:
theta = 2

def p(x: float, th: float) -> float:
    return (th - 1) / (x ** th) if x >= 1 else 0

def F(x: float, th: float) -> float:
    return (1 - x ** (1 - th)) if x >= 1 else 0

def F_inv(y: float, th: float) -> float:
    return (1 - y) ** (1 / (1 - th))

# Размер выборки
N = 100
arr = []

for _ in range(N):
    y = rd.random()
    arr.append(F_inv(y, theta))

Доверительный интервал медианы

$$
P(g(\tilde\theta) - \frac{1.96 * 2^{\frac{1}{\tilde\theta-1}} \ln(2)}{\sqrt{n}(\tilde\theta-1)} < median < g(\tilde\theta) + \frac{1.96 * 2^{\frac{1}{\tilde\theta-1}} \ln(2)}{\sqrt{n}(\tilde\theta-1)})=\beta
$$

$$
P(2 ^ {\overline\ln(x)}- \frac{t_2  2 ^ {\overline\ln(x)} \ln(2) \overline\ln(x)}{\sqrt{n}} < median < 2 ^ {\overline\ln(x)} - \frac{t_1 2 ^ {\overline\ln(x)} \ln(2)  \overline\ln(x)}{\sqrt{n}})=\beta
$$

In [3]:
beta = 0.95

mean_ln = np.mean([np.log(x) for x in arr])

g_theta = 2 ** (mean_ln)

lower_bound = g_theta - 1.96 * g_theta * np.log(2) * mean_ln / np.sqrt(N)
upper_bound = g_theta + 1.96 * g_theta * np.log(2) * mean_ln / np.sqrt(N)

print(f"Доверительный интервал: ({lower_bound}, {upper_bound}), длина: {upper_bound - lower_bound}")

variation_row = sorted(arr)
print("Медиана: ", (variation_row[49] + variation_row[50])/2)

Доверительный интервал: (1.8368907560573788, 2.492066116382171), длина: 0.6551753603247921
Медиана:  2.2257031489009735


Асимптотический доверительный интервал

$$
P(1 - \frac{t_2 - \sqrt{n}}{\overline{\ln(x)} \sqrt{n}} < \theta < 1 - \frac{t_1 - \sqrt{n}}{\overline{\ln(x)} \sqrt{n}} )=\beta
$$

In [4]:
beta = 0.95

lower_bound = 1 - (1.96 - np.sqrt(N)) / (mean_ln * np.sqrt(N))
upper_bound = 1 + (1.96 + np.sqrt(N)) / (mean_ln * np.sqrt(N))

print(f"Доверительный интервал: ({lower_bound}, {upper_bound}), длина: {upper_bound - lower_bound}")

Доверительный интервал: (1.7217109325340898, 2.073589894665139), длина: 0.3518789621310492


#### e)  Численно постройте бутстрапоский доверительный интервал двумя способами, используя параметрический бутстрап и непараметрический бутстрап.

Непараметрический бутстрап

In [5]:
# Количество повторений бутстрапа
n_iterations = 1000
beta = 0.95


# Оценка полученная ОМП
h_wave = 1 + 1 / mean_ln
bootstrap_delta = []

for _ in range(n_iterations):
    sample = np.random.choice(arr, size=N)
    bootstrap_delta.append(1 + 1 / np.mean([np.log(x) for x in sample]) - h_wave)

variation_row = sorted(bootstrap_delta)

t_1 = variation_row[int((1 - beta) * n_iterations / 2)]
t_2 = variation_row[int((1 + beta) * n_iterations / 2)]


lower_bound = h_wave - t_2
upper_bound = h_wave - t_1

print(f"Доверительный интервал: ({lower_bound}, {upper_bound}), длина: {upper_bound - lower_bound}")


Доверительный интервал: (1.7201553820393358, 2.03660782494037), длина: 0.316452442901034


Параметрический бутстрап

In [6]:
# Количество повторений бутстрапа
n_iterations = 50000
beta = 0.95


# Оценка полученная ОМП
h_wave = 1 + 1 / mean_ln
bootstrap_delta = []

for _ in range(n_iterations):
    sample  =  []
    for _ in range(N):
        y = rd.random()
        sample.append(F_inv(y, h_wave))

    bootstrap_delta.append(1 + 1 / np.mean([np.log(x) for x in sample]) - h_wave)

variation_row = sorted(bootstrap_delta)

t_1 = variation_row[int((1 - beta) * n_iterations / 2)]
t_2 = variation_row[int((1 + beta) * n_iterations / 2)]


lower_bound = h_wave - t_2
upper_bound = h_wave - t_1

print(f"Доверительный интервал: ({lower_bound}, {upper_bound}), длина: {upper_bound - lower_bound}")


Доверительный интервал: (1.690587495908206, 2.0509175415531002), длина: 0.36033004564489435


#### f)  Сравнить все интервалы. 

Длина непараметрического бутстраповского дов. инт. < Длина асимптотического дов. инт. < Длина параметрического бутстраповского дов. инт.